In [1]:
# In this tutorial, we will see how to model time series in Python and, then,
# how we can make predictions of future time series data.

# First of all, upload the dataset to Colab (file AirPassengers.csv)

In [ ]:
# Also, install a package that we are going to need

!pip install --user statsmodels -U

In [ ]:
import pandas as pd

dataset = pd.read_csv('AirPassengers.csv', sep=',')
dataset['Month_year'] = pd.to_datetime(dataset['Month'])
dataset['Month'] = dataset['Month_year'].dt.month
dataset = dataset[['Month_year', 'Month', 'Passengers']]
dataset = dataset.set_index('Month_year')

display(dataset)

In [ ]:
# First of all, let us have look at the data

from matplotlib import pyplot as plt

plt.figure(figsize=(20,7))
plt.plot(dataset.index, dataset['Passengers'])
plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")
plt.show()

In [5]:
# The information at out disposal goes from 1949-01-01 to 1960-12-01	
# The dataset covers 144 months, and in fact we have 144 rows in it, one for each month

# From the plot, it immediately comes out that the time series is clearly non-stationary
# The data has a clear upward trend over time
# Moreover, it shows seasonality effects: winter months have typically less passengers than summer ones
# In addition, also observe that variability is increasing with time

In [ ]:
# Let us highlight the trend in our data

import numpy as np


plt.figure(figsize=(20,7))

plt.plot(dataset.index, dataset['Passengers'])

p = np.polyfit(range(len(dataset.index)), dataset['Passengers'], 1) # Least squares polynomial fit: as a result it gives us the parameters a and b, in the line equation y=ax+b
print("p:", p)
plt.plot(dataset.index, [x*p[0] + p[1] for x in range(len(dataset.index))], color='r')

plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")

plt.show()

In [ ]:
# As for the seasonality, we can get confirmation about our thoughts regarding the difference between winter and summer months 
# passenger traffic aggregating the data by month, and making some boxplots.
# Indeed, summer months have more passenger traffic

dataset.boxplot(by='Month', grid=False, figsize=(20,7))


In [ ]:
# Let us try to decompose the series into trend, seasonality and residual
# As we have seen, since in this case the seasonality effects depend on time,
# we have to rely on a multiplicative model to do that, i.e., Value = Trend * Seasonality * Error

from statsmodels.tsa.seasonal import seasonal_decompose

result_mul = seasonal_decompose(dataset['Passengers'], model='multiplicative')

fig = result_mul.plot()
fig.set_size_inches((15, 7))
fig.tight_layout()
plt.show()

In [ ]:
# Now, let us make some forecasts!

# The first step is making the time series stationary.
# Differencing is a classical operation by which we can do that.
# Differencing can help to stabilize the mean of the time series by removing changes in the level
# of a time series, and so eliminating (or reducing) trend and seasonality effects.
# Differencing is performed by subtracting the previous observation from the current observation.

# Pandas already provides the diff() method that does exactly what we would like to do

differenced_series = dataset['Passengers'].diff()


plt.figure(figsize=(20,7))

plt.plot(dataset.index, differenced_series)

plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")

plt.show()

# Of course, here we have lost the first value of the series, since it does not have any point wrt calculate its difference

# Anyway, the result is clearly still not stationary... what happened here?

In [ ]:
# The reason is that, since our time series has a strong seasonality effect, to a given value of index "i", we should
# subtract the value at index "i-[length of the period]", istead of the one at "i-1"

# To get a confirmation of that, let us investigate the autocorrelation of the time series

plt.figure(figsize=(12,7))
pd.plotting.autocorrelation_plot(dataset['Passengers'])
plt.xlim(0, 120) # let us show the result up to 120 lags, i.e., 10 years
plt.xticks(range(0,120,6))
plt.show()

# Notice how the autocorrelation decreases with the increase of the lag value
# Nevertheless, we can clearly see the seasonal peaks at lags 12, 24, 36, 48 
# Thus, our period here is 12 (unsurprisingly, a year)

In [ ]:
# Here we subtract from each value of the time series the value that was observed in the same season one year earlier

differenced_series = dataset['Passengers'].diff(12) # here, we are not considering the previous value anymore for the subtraction, but the 12th predecessor (the step consumes the initial 12 values)

plt.figure(figsize=(20,7))

plt.plot(dataset.index, differenced_series)

plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")

plt.show()

# Uhm, better... but the time series is still not stationary, there is still a clear upward trend here...

In [ ]:
# Let us now apply classical differencing on the seasonally differenced time series

differenced_series = differenced_series.diff()


plt.figure(figsize=(20,7))

plt.plot(dataset.index, differenced_series)

plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")

plt.show()

# Now, this is stationary!

# Observe that differencing can be applied several times, until we reach a stationary time series

# As a final note, observe that the differencing process is invertible.
# The inverse difference is the cumulative sum of the first value of the original series and then subsequent differences

In [ ]:
# Let us turn to some prediction tasks
# The first step is that of dividing our data into a training and a test set

train = dataset[dataset.index < pd.to_datetime("1957-12", format='%Y-%m')]
test = dataset[dataset.index >= pd.to_datetime("1957-12", format='%Y-%m')]


# First of all, let us have look at the data

plt.figure(figsize=(20,7))
plt.plot(dataset.index, dataset['Passengers'])
plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")
plt.axvline(pd.to_datetime("1957-12", format='%Y-%m'), color='r')
plt.show()


In [ ]:
# Our baseline will be provided by a rather naive methodology, i.e., as the prediction we take the last known value

prediction = list(train['Passengers'][-1:])*len(test.index)

plt.figure(figsize=(20,7))
plt.plot(train.index, train['Passengers'], color='tab:blue', label='training_data')
plt.plot(test.index, test['Passengers'], linestyle='--', color='tab:blue', label='ground_truth_test')
plt.plot(test.index, prediction, color='tab:orange', label='prediction')
plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")
plt.legend()
plt.show()

from sklearn.metrics import mean_squared_error

print("Root mean squared error:", np.sqrt(mean_squared_error(test["Passengers"], prediction)))

In [ ]:
# Now we will employ (S)ARIMA 
# ARIMA stands for Auto Regressive Integrated Moving Average.
# The acronym indicates that a ARIMA model has three components, i.e.:
# - Auto-Regressive: past time series points may impact its current and future values. Thus, ARIMA uses p past observations
#                    to forecast current and future values. Auto-regression is employed, that is, the process of applying
#                    regression on a variable considering past values of itself. In other words, the current value of the series
#                    is determined as a linear combination of p past values (which may also be previous forecasts).
# - Integrated: the forecasting method cannot be applied to non-stationary time series. Thus, differencing is applied to 
#               reduce trend and seasonality effects.
# - Moving Average: also past noise in the series might influence its current and future values. For instance, a "shock" in a 
#                   stock market time series will persist in a smaller extent in the next days as well. Thus, rather than
#                   using the past values of the forecast, the moving average model uses past forecast errors in a
#                   regression-like fashion. Specifically, the result is a weighted moving average over past forecast errors.
# Specifically, since our data is seasonal, we will employ SARIMA (Auto Regressive Integrated Moving Average)
#
#
# We will rely on the function "ARIMA of the package "statsmodels"
#
# The function call is as follows: ARIMA(ts, order=(0, 0, 0), seasonal_order=(0, 0, 0, 0))
#  - ts: is the training time series
#  - order expects three values (p, d, q):
#      p: number of lag values to consider in the auto-regressive part  (e.g., p=3 means we will use the 3 previous values of
#           our time series in the autoregressive portion of the calculation)
#      d: number of differencing transformations applied to the time series to make it stationary (from previous experiments,
#           we already know we need 1 seasonal differencing and 1 "plain" differencing)
#      q: the size of the moving average window
#  - seasonal order expects four values (p, d, q, s): p, d, q are as before but related to the seasonal part, while "s" is the length of the period
#
# Thus, ARIMA is capable of dealing with time series differencing on its own (as a difference with respect to ARMA)

from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(train['Passengers'], order=(3, 1, 0), seasonal_order=(2, 1, 0, 12)) # using original training data and instructing ARIMA how to perform differencing
model = model.fit()
print(model.summary())

In [ ]:
# And now let us use the model for prediction purposes

prediction = model.forecast(37)

plt.figure(figsize=(20,7))
plt.plot(train.index, train['Passengers'], color='tab:blue', label='training_data')
plt.plot(test.index, test['Passengers'], linestyle='--', color='tab:blue', label='ground_truth_test')
plt.plot(test.index, prediction, color='tab:orange', label='prediction')
plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")
plt.legend()
plt.show()

print("Root mean squared error:", np.sqrt(mean_squared_error(test["Passengers"], prediction)))

In [ ]:
# Now let us try a RandomForestRegressor to perform the prediction
# This is a fundamentally different approach with respect to the previous one, since
# this kind of model is not capable of performing extrapolation from the training set data, i.e.,
# it is only capable of handling values that it has seen during the training phase.
# Here, we will understand even better why stationarizing a time series is important


n_lags = 3

# The following function generates a dataset in the format required by the RandomForestRegressor

def buildLaggedFeatures(input_series, lag=3):
  result = np.zeros((len(input_series)-lag, lag+1))
  for i in range(lag, len(input_series)):
    result[i-lag] = [input_series[i-j] for j in range(lag, -1, -1)]
  colnames = ["lag_" + str(i) for i in range(lag, 0, -1)]
  return pd.DataFrame(result[:,:-1], columns=colnames), pd.DataFrame(result[:,-1], columns=["Label"])


X_train, y_train = buildLaggedFeatures(train['Passengers'], lag=n_lags)
X_test, y_test = buildLaggedFeatures(test['Passengers'], lag=n_lags)


display(X_train)
display(y_train)

display(train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=42)
model.fit(X_train.values, y_train)


# Of course, when we perform the prediction, we start from the last known values of the training set
# Then, each successive predictions will be based on the previous predictions
# E.g.: [train_n-2, train_n-1, train_n] --> pred_0, [train_n-1, train_n, pred_0] --> pred_1, [train_n, pred_0, pred_1] --> pred_2, ...

# input: list of training labels, number of lag values to consider, number of predictions to perform
def rf_predict(train_labels, lag, num_preds, model_in):
  predictors = train_labels[-lag:].values.flatten()
  preds = []
  for i in range(num_preds):
    pred = model_in.predict(predictors.reshape(-1, lag))
    preds.append(pred)
    predictors = np.roll(predictors, -1)
    predictors[-1] = pred
  return preds



# And now let us use the model for prediction purposes

prediction = rf_predict(y_train, n_lags, 37, model)

plt.figure(figsize=(20,7))
plt.plot(train.index, train['Passengers'], color='tab:blue', label='training_data')
plt.plot(test.index, test['Passengers'], linestyle='--', color='tab:blue', label='ground_truth_test')
plt.plot(test.index, prediction, color='tab:orange', label='prediction')
plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")
plt.legend()
plt.show()

print("Root mean squared error:", np.sqrt(mean_squared_error(test["Passengers"], prediction)))


# Note that RF cannot predict the future values very well
# This is to be expected, as the training set provided no information about how to estimate them
# Specificially we could not predict the increase in the trend and seasonality effect just based on the 3 previous lag values

In [ ]:
# Generating training and test datasets introducing also the differencing operations

differenced_series = dataset['Passengers'].diff(12) # differencing taking into account seasonality
differenced_series = differenced_series.diff() # another step of differencing

train_differenced = differenced_series[dataset.index < pd.to_datetime("1957-12", format='%Y-%m')].dropna() # the first 13 values are undefined due to the differencing operation
test_differenced = differenced_series[dataset.index >= pd.to_datetime("1957-12", format='%Y-%m')]

X_train, y_train = buildLaggedFeatures(train_differenced, lag=n_lags)
X_test, y_test = buildLaggedFeatures(test_differenced, lag=n_lags)


# Training model
model = RandomForestRegressor(random_state=42)
model.fit(X_train.values, y_train)


# And now let us use the model for prediction purposes

prediction = rf_predict(y_train, n_lags, 37, model)


# Helper function that we are gointo to use to revert the differencing steps
def diff_inv(train_vals, series_in, difflag=1):
  whole_series = list(train_vals) + list(series_in)
  for i in range(0, len(whole_series)):
    if i - difflag >= 0:
      whole_series[i] = whole_series[i] + whole_series[i-difflag]
  return whole_series


revert_diff = diff_inv(np.asarray(y_train).flatten(), np.asarray(prediction).flatten(), 1)
revert_season = diff_inv(np.asarray(y_train).flatten(), revert_diff, 12)
revert_initial_value = np.asarray(revert_season) + train['Passengers'][12]
actual_prediction = revert_initial_value[-37:]

plt.figure(figsize=(20,7))
plt.plot(train.index, train['Passengers'], color='tab:blue', label='training_data')
plt.plot(test.index, test['Passengers'], linestyle='--', color='tab:blue', label='ground_truth_test')
plt.plot(test.index, actual_prediction, color='tab:orange', label='prediction')
plt.title("Number of passengers by month")
plt.xticks(dataset.index[::3], rotation=90)
plt.xlabel("Month_year")
plt.ylabel("Number of passengers")
plt.legend()
plt.show()

print("Root mean squared error:", np.sqrt(mean_squared_error(test_differenced.values, prediction)))